# Anomaly Detection Methods - Comparison Template

## Purpose
This notebook provides a clean template for developing and comparing anomaly detection methods.

## Structure
1. **Configuration** - Imports and global settings
2. **Data Loading** - Load and preprocess datasets
3. **Data Preparation** - Create train/test splits
4. **Detection Methods** - Implement your methods here
5. **Benchmarking** - Compare all methods
6. **Results & Visualization** - Analyze and visualize results

## How to Use
1. Run the **Configuration** and **Data Loading** sections first
2. Implement your detection method in the **Detection Methods** section
3. Add your method to the `MODELS` dictionary in the **Benchmarking** section
4. Run the benchmarking and visualization sections

## Notes
- Each detection method should implement a `predict(X) -> np.ndarray` function
- Predictions should return binary labels: 0 (normal) or 1 (anomaly)
- Keep method implementations in separate sections for clarity

## 1. Configuration

### 1.1 Imports

In [ ]:
# Evaluation and comparison tools
from lib.evaluation.ModelComparisonAnalyzer import ModelComparisonAnalyzer
from lib.evaluation.model_evaluator import evaluate_models

# Data utilities
from lib.data.data_generator import generate_synthetic_datasets, load_datasets

# Standard libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import display

import matplotlib.pyplot as plt
import numpy as np
from typing import Any, Dict, List, Literal, Optional, Tuple
from pathlib import Path
import pandas as pd

from numpy.typing import NDArray

print("✅ All imports successful!")

### 1.2 Global Constants

In [ ]:
# Column names
TARGET_VARIABLE_COLUMN_NAME = "faultNumber"
SIMULATION_RUN_COLUMN_NAME = "simulationRun"
COLUMNS_TO_REMOVE = [TARGET_VARIABLE_COLUMN_NAME, SIMULATION_RUN_COLUMN_NAME, "sample"]

# Fault injection point (where faults start in the test data)
FAULT_INJECTION_POINT = 160

# Data source: 'real' (load RData files) or 'synthetic' (lightweight random data)
DATA_SOURCE: Literal["real", "synthetic"] = "synthetic"

# Synthetic data configuration (used when DATA_SOURCE = 'synthetic')
SYNTHETIC_DATA_CONFIG: Dict[str, Any] = {
    "n_simulation_runs": 5,
    "n_faults": 5,
    "n_samples": 300,
    "n_features": 52,
    "random_state": 42,
    "feature_prefix": "feature",
}

print("✅ Configuration set!")

## 2. Data Loading

### 2.1 Load Raw Datasets

In [ ]:
# Load datasets
datasets = load_datasets(DATA_SOURCE, synthetic_config=SYNTHETIC_DATA_CONFIG)

DF_FF_TRAINING_RAW = datasets["fault_free_training"]
DF_FF_TEST_RAW = datasets["fault_free_testing"]
DF_F_TRAINING_RAW = datasets["faulty_training"]
DF_F_TEST_RAW = datasets["faulty_testing"]

print(f"Data source: {DATA_SOURCE}")
print(f"Fault-free training shape: {DF_FF_TRAINING_RAW.shape}")
print(f"Fault-free test shape: {DF_FF_TEST_RAW.shape}")
print(f"Faulty training shape: {DF_F_TRAINING_RAW.shape}")
print(f"Faulty test shape: {DF_F_TEST_RAW.shape}")

### 2.2 Data Preview

In [ ]:
# Preview the data structure
print("Fault-free training data (first 3 rows):")
display(DF_FF_TRAINING_RAW.head(3))

print("\nFaulty training data (first 3 rows):")
display(DF_F_TRAINING_RAW.head(3))

print("\nUnique faults:", sorted(DF_F_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME].unique()))
print("Unique simulations:", sorted(DF_F_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME].unique()))

## 3. Data Preparation

### 3.1 Fit Scaler on Full Training Data

In [ ]:
# Fit scaler on all fault-free training data (in-control)
scaler = StandardScaler()
X_INCONTROL_TRAIN_FULL_DF = DF_FF_TRAINING_RAW.drop(columns=COLUMNS_TO_REMOVE, axis=1)

scaler.fit(X_INCONTROL_TRAIN_FULL_DF)
X_INCONTROL_TRAIN_FULL_SCALED = scaler.transform(X_INCONTROL_TRAIN_FULL_DF)

print("In-control training data (full) shape:", X_INCONTROL_TRAIN_FULL_SCALED.shape)
print("Number of features:", X_INCONTROL_TRAIN_FULL_SCALED.shape[1])

### 3.2 Create Single Simulation Run Datasets

In [ ]:
# Select a specific simulation run and fault for quick testing
simulation_run = 1
fault_number = 2

# Filter data by simulation run
DF_FF_TRAINING = DF_FF_TRAINING_RAW.query("simulationRun == @simulation_run")
DF_FF_TEST = DF_FF_TEST_RAW.query("simulationRun == @simulation_run")
DF_F_TRAINING = DF_F_TRAINING_RAW.query("faultNumber == @fault_number and simulationRun == @simulation_run")
DF_F_TEST = DF_F_TEST_RAW.query("faultNumber == @fault_number and simulationRun == @simulation_run")

# Prepare feature matrices
X_INCONTROL_TRAIN_REDUCED_DF = DF_FF_TRAINING.drop(columns=COLUMNS_TO_REMOVE, axis=1)
X_INCONTROL_TEST_REDUCED_DF = DF_FF_TEST.drop(columns=COLUMNS_TO_REMOVE, axis=1)
X_OUT_OF_CONTROL_TEST_REDUCED_DF = DF_F_TEST.drop(columns=COLUMNS_TO_REMOVE, axis=1)
X_OUT_OF_CONTROL_TRAIN_REDUCED_DF = DF_F_TRAINING.drop(columns=COLUMNS_TO_REMOVE, axis=1)

# Scale data
X_INCONTROL_TRAIN_REDUCED_SCALED = scaler.transform(X_INCONTROL_TRAIN_REDUCED_DF)
X_INCONTROL_TEST_REDUCED_SCALED = scaler.transform(X_INCONTROL_TEST_REDUCED_DF)
X_OUT_OF_CONTROL_TEST_REDUCED_SCALED = scaler.transform(X_OUT_OF_CONTROL_TEST_REDUCED_DF)
X_OUT_OF_CONTROL_TRAIN_REDUCED_SCALED = scaler.transform(X_OUT_OF_CONTROL_TRAIN_REDUCED_DF)

print(f"Using simulation run: {simulation_run}, fault: {fault_number}")
print(f"In-control training (reduced): {X_INCONTROL_TRAIN_REDUCED_SCALED.shape}")
print(f"In-control test (reduced): {X_INCONTROL_TEST_REDUCED_SCALED.shape}")
print(f"Out-of-control training: {X_OUT_OF_CONTROL_TRAIN_REDUCED_SCALED.shape}")
print(f"Out-of-control test: {X_OUT_OF_CONTROL_TEST_REDUCED_SCALED.shape}")

### 3.3 Cut Data After Fault Injection Point

In [ ]:
# Remove samples before fault injection point
X_INCONTROL_TEST_REDUCED_SCALED_CUT = X_INCONTROL_TEST_REDUCED_SCALED[FAULT_INJECTION_POINT:]
X_OUT_OF_CONTROL_TEST_REDUCED_SCALED_CUT = X_OUT_OF_CONTROL_TEST_REDUCED_SCALED[FAULT_INJECTION_POINT:]

print(f"Fault injection point: {FAULT_INJECTION_POINT}")
print(f"In-control test (cut): {X_INCONTROL_TEST_REDUCED_SCALED_CUT.shape}")
print(f"Out-of-control test (cut): {X_OUT_OF_CONTROL_TEST_REDUCED_SCALED_CUT.shape}")

### 3.4 Summary of Available Datasets

After running the cells above, you have access to:

**Full datasets (all simulation runs):**
- `X_INCONTROL_TRAIN_FULL_SCALED` - All fault-free training data (scaled)
- `DF_FF_TEST_RAW`, `DF_F_TEST_RAW` - Raw test data (for benchmarking)

**Single simulation run datasets:**
- `X_INCONTROL_TRAIN_REDUCED_SCALED` - In-control training
- `X_INCONTROL_TEST_REDUCED_SCALED` - In-control test
- `X_OUT_OF_CONTROL_TEST_REDUCED_SCALED` - Out-of-control test

**Cut datasets (after fault injection):**
- `X_INCONTROL_TEST_REDUCED_SCALED_CUT` - In-control test (cut)
- `X_OUT_OF_CONTROL_TEST_REDUCED_SCALED_CUT` - Out-of-control test (cut)

**Scaler:**
- `scaler` - Fitted StandardScaler for transforming new data

## 4. Detection Methods

### 📝 Instructions for Adding New Methods

Each detection method should:
1. Be implemented in its own subsection (e.g., `### 4.1 My Method`)
2. Train on `X_INCONTROL_TRAIN_FULL_SCALED` (or reduced version)
3. Define a prediction function that takes `X: np.ndarray` and returns binary predictions
4. Be added to the `MODELS` dictionary in Section 5

**Example structure:**
```python
# Train your method
my_detector = MyDetector()
my_detector.fit(X_INCONTROL_TRAIN_FULL_SCALED)

# Create prediction wrapper
def my_detector_predict(X: np.ndarray) -> np.ndarray:
    return my_detector.predict(X)  # Returns 0/1 labels

# Quick test
predictions = my_detector_predict(X_INCONTROL_TEST_REDUCED_SCALED_CUT)
arl = np.argmax(predictions == 1) if np.any(predictions == 1) else None
print(f"ARL (first detection): {arl}")
```

### 4.1 Method Template (Example)

In [ ]:
# TODO: Replace this with your actual detection method

# Example: Simple threshold-based detector
class ExampleDetector:
    def __init__(self, threshold_percentile=95):
        self.threshold_percentile = threshold_percentile
        self.threshold = None
        self.mean = None
    
    def fit(self, X):
        """Fit on in-control training data."""
        self.mean = np.mean(X, axis=1)
        self.threshold = np.percentile(self.mean, self.threshold_percentile)
        return self
    
    def predict(self, X):
        """Predict anomalies (0 = normal, 1 = anomaly)."""
        scores = np.mean(X, axis=1)
        return (scores > self.threshold).astype(int)

# Train detector
example_detector = ExampleDetector(threshold_percentile=95)
example_detector.fit(X_INCONTROL_TRAIN_FULL_SCALED)

# Create prediction wrapper
def example_detector_predict(X: np.ndarray) -> np.ndarray:
    return example_detector.predict(X)

# Quick test
predictions_normal = example_detector_predict(X_INCONTROL_TEST_REDUCED_SCALED_CUT)
predictions_anomaly = example_detector_predict(X_OUT_OF_CONTROL_TEST_REDUCED_SCALED_CUT)

arl0 = np.argmax(predictions_normal == 1) if np.any(predictions_normal == 1) else None
arl1 = np.argmax(predictions_anomaly == 1) if np.any(predictions_anomaly == 1) else None

print(f"Example Detector - ARL0 (In-control): {arl0}")
print(f"Example Detector - ARL1 (Out-of-control): {arl1}")

### 4.2 Your Method 1

Add your first detection method here.

In [ ]:
# TODO: Implement your first detection method
#
# # Train your method
# method1 = YourMethod1()
# method1.fit(X_INCONTROL_TRAIN_FULL_SCALED)
#
# # Create prediction wrapper
# def method1_predict(X: np.ndarray) -> np.ndarray:
#     return method1.predict(X)
#
# # Quick test
# predictions = method1_predict(X_INCONTROL_TEST_REDUCED_SCALED_CUT)
# arl = np.argmax(predictions == 1) if np.any(predictions == 1) else None
# print(f"Method 1 - ARL: {arl}")

pass  # Remove this when you add your code

### 4.3 Your Method 2

Add your second detection method here.

In [ ]:
# TODO: Implement your second detection method

pass  # Remove this when you add your code

### 4.4 Your Method 3

Add additional methods as needed...

In [ ]:
# TODO: Add more methods as needed

pass  # Remove this when you add your code

## 5. Benchmarking & Comparison

### 5.1 Define Models Dictionary

Add all your detection methods to the `MODELS` dictionary below.
Each entry should map a method name to its prediction function.

In [ ]:
# Define all models to compare
MODELS = {
    "ExampleDetector": example_detector_predict,
    # TODO: Add your methods here
    # "Method1": method1_predict,
    # "Method2": method2_predict,
    # "Method3": method3_predict,
}

print(f"Models to compare: {list(MODELS.keys())}")

### 5.2 Run Evaluation

In [ ]:
# Run evaluation on all simulation runs and faults
df_results, summary_df = evaluate_models(
    models=MODELS,
    df_ff_test_raw=DF_FF_TEST_RAW,
    df_f_test_raw=DF_F_TEST_RAW,
    scaler=scaler,
    fault_injection_point=FAULT_INJECTION_POINT,
    simulation_column=SIMULATION_RUN_COLUMN_NAME,
    fault_column=TARGET_VARIABLE_COLUMN_NAME,
    columns_to_remove=COLUMNS_TO_REMOVE,
    verbose=True
)

print("\n" + "="*80)
print("EVALUATION COMPLETE")
print("="*80)
print(f"Results shape: {df_results.shape}")
print(f"Summary shape: {summary_df.shape}")
print(f"Models evaluated: {', '.join(MODELS.keys())}")

### 5.3 Initialize Analyzer

In [ ]:
# Initialize the model comparison analyzer
analyzer = ModelComparisonAnalyzer(df_results, summary_df)
print("✅ Analyzer initialized!")

## 6. Results & Visualization

### 6.1 ARL Analysis

In [ ]:
# Plot ARL (Average Run Length) analysis
analyzer.plot_arl_analysis()
plt.show()

### 6.2 Average ARL Performance

In [ ]:
# Plot average ARL across all faults
analyzer.plot_arl_averages()
plt.show()

### 6.3 Overall Classification Metrics

In [ ]:
# Plot overall classification metrics (Precision, Recall, F1, etc.)
analyzer.plot_overall_metrics()
plt.show()

**Metrics explanation:**
- **Precision**: High when few false alarms (normal data flagged as anomaly)
- **Recall**: High when most anomalies are caught (few missed detections)
- **Specificity**: High when normal data is correctly identified
- **Accuracy**: High when few mistakes overall
- **F1 Score**: Harmonic mean of precision and recall

### 6.4 Per-Fault Classification Metrics

In [ ]:
# Plot metrics for each individual fault
analyzer.plot_per_fault_bars()
plt.show()

### 6.5 False Positive and False Negative Rates

In [ ]:
# Plot FPR and FNR (lower is better for both)
analyzer.plot_false_rates()
plt.show()

### 6.6 Confusion Matrix - Average

In [ ]:
# Average confusion matrix across all faults
analyzer.plot_confusion_matrix_average()
plt.show()

### 6.7 Confusion Matrix - Per Fault

In [ ]:
# Confusion matrix for each fault
analyzer.plot_confusion_matrices_per_fault()
plt.show()

### 6.8 Performance Heatmaps

In [ ]:
# Heatmaps showing performance across faults
analyzer.plot_heatmaps()
plt.show()

### 6.9 Statistical Distributions

In [ ]:
# Distribution plots for various metrics
analyzer.plot_distributions()
plt.show()

## 7. Summary & Conclusions

### Summary Tables

Use the cells below to display summary tables and export results.

In [ ]:
# Display summary statistics
print("Summary Statistics:")
display(summary_df.describe())

print("\nMean metrics by model:")
display(summary_df.groupby('model').mean())

### Export Results

Optionally export results to CSV for further analysis.

In [ ]:
# Uncomment to export results
# df_results.to_csv('results/detailed_results.csv', index=False)
# summary_df.to_csv('results/summary_results.csv', index=False)
# print("✅ Results exported!")

### Notes

Add your conclusions and observations here:

- ...
- ...
- ...